In [1]:
from topologicpy.Graph import Graph
from topologicpy.Helper import Helper
import topologicpy
from topologicpy.Topology import Topology
from topologicpy.Vertex import Vertex
from topologicpy.Wire import Wire
from topologicpy.Face import Face
from topologicpy.Shell import Shell
from topologicpy.Cell import Cell
from topologicpy.Dictionary import Dictionary
from topologicpy.Cluster import Cluster
from src.topologicpy.Edge import Edge
from src.topologicpy.CellComplex import CellComplex

print(Helper.Version())
print(topologicpy.__file__)
renderer = "browser"

/opt/anaconda3/envs/eco-topologicpy/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The version that you are using (0.8.15) is the latest version available on PyPI.
/opt/anaconda3/envs/eco-topologicpy/lib/python3.8/site-packages/topologicpy/__init__.py


# Cube Grid Function

In [2]:
building_complex = CellComplex.Box(width=3, length=3, height=3, uSides=3, vSides=3, wSides=3)
print(building_complex)

#Topology.Show(building_complex, renderer=renderer)

In [3]:
from topologicpy.Topology import Topology
from topologicpy.CellComplex import CellComplex
from topologicpy.Vertex import Vertex
from topologicpy.Face import Face


def create_environment_shell(building_complex):
    """
    Creates an environment shell centered around `building_complex` with a **horizontal separation**.
    Returns both the repositioned `building_complex` and `env_shell`.

    Parameters:
        building_complex (topologic_core.CellComplex): The building.

    Returns:
        tuple: (repositioned_building_complex, env_shell)
    """
    width, length, height = 4.0, 4.0, 4.0  

    lower_ratio = 0.3  
    upper_ratio = 1.0 - lower_ratio  

    lower_height = height * lower_ratio
    upper_height = height * upper_ratio

    building_bb = Topology.BoundingBox(building_complex)
    bb_vertices = Topology.Vertices(building_bb)

    min_z = min(Vertex.Z(v) for v in bb_vertices)
    max_z = max(Vertex.Z(v) for v in bb_vertices)
    building_center = Topology.Centroid(building_complex)

    # Compute the vertical center shift
    building_height = max_z - min_z
    building_center_z = (max_z + min_z) / 2 
    env_bottom = building_center_z - height / 2

    building_offset_z = -building_center_z 
    repositioned_building_complex = Topology.Translate(building_complex, 0, 0, building_offset_z)

    env_full = CellComplex.Box(
        width=width, length=length, height=height,
        uSides=1, vSides=1, wSides=1 
    )

    env_offset_z = env_bottom - min(Vertex.Z(v) for v in Topology.Vertices(env_full))
    env_complex = Topology.Translate(env_full, 0, 0, env_offset_z)

    split_height = env_bottom + lower_height
    split_face = Face.Rectangle(
        origin=Vertex.ByCoordinates(0, 0, split_height),
        width=width * 1.1, length=length * 1.1
    )

    sliced_env = Topology.Slice(env_complex, split_face)

    env_cells = CellComplex.Cells(sliced_env)
    if not env_cells:
        print("⚠️ Error: No cells found after slicing. Returning original environment shell.")
        return repositioned_building_complex, env_complex

    env_shell = Topology.Difference(sliced_env, repositioned_building_complex)

    return repositioned_building_complex, env_shell

In [4]:
building_complex, env_complex = create_environment_shell(building_complex)


building_graph = Graph.ByTopology(building_complex, viaSharedTopologies=True, toExteriorTopologies=True)
env_graph = Graph.ByTopology(env_complex, toExteriorTopologies=True)

print(env_complex)
print(building_complex)
print(building_graph)
print(env_graph)

#Topology.Show(env_complex, env_graph, renderer=renderer)
#Topology.Show(building_complex, building_graph, renderer=renderer)

In [21]:
merged = Topology.Merge(building_complex, env_complex)
m_graph = Graph.ByTopology(merged, useInternalVertex=True, viaSharedTopologies=True)

In [22]:
Topology.Show(merged, m_graph, renderer=renderer, sagitta=0.05, faceOpacity=0.1, absolute=False,
              vertexSizeKey="closeness_centrality", backgroundColor="white")

# TODO: Brauchen eigene Graph Funktion basierend auf Centroids oder in Wand (Boden / Aussen) als Zonen speichern?

In [23]:
Graph.PyvisGraph(m_graph, "/Users/ecolutionengineering/PycharmProjects/eco-topologicpy/ecolution/notebooks")

/Users/ecolutionengineering/PycharmProjects/eco-topologicpy/ecolution/notebooks.html
